### 【総合演習３Extra】賃貸物件情報から賃貸価格を予測してみよう

総合演習③は番外編です。  
総合演習①では、入手したデータに対して基礎分析を行うために必要なデータ加工を学びました。  
総合演習②では、加工したデータに対してデータ概要を把握するための集計や可視化を学びました。  
この総合演習③でほんの少しだけ、加工したデータを使って予測するモデルを作る体験をしていきます。  

総合演習③の内容は、Pythonデータ加工研修の学習範囲外となります。  
また総合演習①および②についても今回は実装方法を学習のメインとしていますが、実際は統計的知見も必要とします。  
これらの統計的知見については、「（Rによる）データ解析基礎研修」で取り扱っておりますので興味のある方は是非ご参加ください。  

【総合演習の取り組み方】  
* この演習は全てのコードが入力済みとなります。解説を読みながら実行してください。
* あくまで加工したデータを使って実際に予測を体験してみることが目的となりますので、詳細な解説等は省略されています。

---
#### 【Extra0】賃貸価格を予測する
---

手元にあるデータを用いて分析を行うモチベーションは主に以下の2つがあります。
* どのような要素が目的となる項目（ターゲット）に影響しているのか把握したい（要因）
* 目的となる項目（ターゲット）を他の項目を使って推定したい（予測）

今回の総合演習で扱っているデータは山手線沿線の賃貸物件のデータです。  
面積や最寄り駅までの距離、階数など様々な要素が賃貸価格に影響しています。  

ここからはこの賃貸物件のデータを用いて、賃貸価格の予測にチャレンジしていきます。

---
#### 【Extra1】ライブラリインポートとデータの読み込み
---

まずはデータ加工や可視化に必要なライブラリをインポートします。  
また可視化のmatplotlibの日本語の文字化け対応および、カラムの最大表示数の変更を行います。

In [1]:
# ライブラリインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# matplotlibの日本語文字化け対応
import matplotlib
font = {'family':'Yu Mincho'}
matplotlib.rc('font', **font)  

In [3]:
# 最大表示数の指定（ここでは100行を指定）   
pd.set_option('display.max_columns', 100)
pd.get_option("display.max_columns")

100

総合演習①で作成したchintaiデータを読み込みます。  
※ここではJpyter Notebookと同じディレクトリにchintai.csvが配置されている指定となります。必要に応じて修正してください。

In [4]:
# chintai.csv読込
chintai = pd.read_csv('chintai.csv')

chintai.head(2)

,物件ID,建物ID,物件階,間取り,面積,家賃,管理費,敷金,礼金,平米単価,住所1,住所2,住所3,築年数,地上階,地下,沿線1,駅名1,駅徒歩1,沿線2,駅名2,駅徒歩2,沿線3,駅名3,駅徒歩3,駅徒歩最短,山手線駅,山手線駅徒歩
0,R0000001,B0000001,2,44K,552.58,2484000.0,0,0.0,1.0,4495.276702,東京都,北区,田端,32,6,1,ＪＲ山手線,田端駅,6,ＪＲ山手線,西日暮里駅,12.0,東京メトロ千代田線,千駄木駅,16.0,6.0,田端駅,6.0
1,R0000002,B0000002,7,1K,26.10,100000.0,8000,1.0,1.0,3831.417625,東京都,台東区,上野,6,14,0,ＪＲ山手線,秋葉原駅,8,ＪＲ山手線,御徒町駅,5.0,東京メトロ日比谷線,仲御徒町駅,2.0,2.0,御徒町駅,5.0


---
#### 【Extra2】単回帰分析（面積だけで家賃を予測する）をやってみる
---

物件の条件から「家賃」を予測したいと考えました。今回は「面積」の1項目を使って予測を行います。  
データから予測するというのは、手元にある過去のデータを学習したモデルを作成し、そのモデルを使って予測するということです。  

今回はモデルの入門として王道の回帰分析を使って体験してみましょう。

●回帰分析とは●  
一般的には線形回帰が利用され、以下のような式で表されます。  

$ y = b_0 + b_1x_1 + b_2x_2 + b_3x_3 + … + b_kx_k + \epsilon $

$ y $：目的変数   
$ b_0 $：切片  
$ b_1 ～ b_k $ ：係数（傾き）  
$ x_1 ～ x_k $ ：説明変数  
$ /epsilon $ ：誤差（モデルでは予測できなかった誤差）

まずは単回帰分析で実施してみようと思います。  
単回帰分析は、$ y = b_0 + b_1x + \epsilon $で表されます。  

今回は家賃を面積だけで予測するので   
$家賃 = b * 面積 + 切片(面積0でも発生する金額)$ というモデルを作成します。

##### _１）データの分割とモデルの作成_

Pythonでモデルを実装する際、よく使用されるライブラリに`scikit-learn`（sklearn）があります。  
複雑な数式が必要となるモデル実装もライブラリを使用することで手軽に実装することができます。  
今回使用するscikit-learnの関数をインポートします。  

In [ ]:
# 機械学習ライブラリscikit-learnの線形回帰をインポート
from sklearn.linear_model import LinearRegression
# 機械学習ライブラリscikit-learnの学習/検証データ分割をインポート
from sklearn.model_selection import train_test_split

まずモデル作成に必要となるデータを作成していきます。  
今回のモデル作成には目的変数Y（予測したい項目）と説明変数X（予測したい項目を説明する項目）が必要となります。  

ここでは家賃を面積で予測するモデルを作成するので、Yに家賃をXに面積を指定します。  

In [ ]:
# 説明変数をNumpy配列で格納
X = chintai[['面積']].values
# 目的変数をNumpy配列で格納
Y = chintai['家賃'].values

モデルを作成する際に使用するデータは学習データと検証データに分けておきます。  
全部のデータを使ってモデルを作ってしまうと、本当にそのモデルが未知のデータに対して正しく予測できるか証明できません。  
モデルを作成するための学習データ(train)と、作成したモデルが正しく予測できてるか確認するための検証データ(test)に分割します。

In [ ]:
# 学習データと検証データを7:3で分割
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.7, test_size=0.3, random_state=0)

学習用データを使用して、モデルを作成します。  
線形回帰のモデルのインスタンスを生成し、`fit(説明変数x, 目的変数y)`で学習を行います。

In [ ]:
# 線形回帰のインスタンスを生成する
clf = LinearRegression()

# 学習データから学習してモデルを作る
clf.fit(X_train, Y_train)

In [ ]:
# 作成されたモデル式を確認する
print('モデル関数の回帰係数 : %.3f' %clf.coef_)
print('モデル関数の切片 : %.3f' %clf.intercept_)

print('y= %.3fx + %.3f' % (clf.coef_ , clf.intercept_))

学習を行うとインスタンス（作成されたモデル）に、様々情報が作られます。  
上記では回帰式の係数である回帰係数（傾き）と切片を出力しています。  
この値から `y = モデルの回帰係数 * x + モデルの切片` を導き出すことができます。

##### _２）作成したモデルを検証データで予測して精度を確認する_

作成したモデルを使って、検証データを予測してみましょう。  
予測は`predict(説明変数x)`で行います。

In [ ]:
# 検証データをモデルで予測する
Y_pred = clf.predict(X_test)

予測した結果（予測値）について、実際の値（実測値）と比べてみましょう。  
今回は先頭4つの値を比べてみます。  

In [ ]:
# 予測した値と実際の値を比べてみる
print('予測値：', Y_pred[:4])
print('実測値：', Y_test[:4])

良い線をいっている予測値もあれば大幅に外している予測値もあります。  
作成したモデルは果たして良いモデルなのでしょうか。  
良いモデルかどうかの判断は様々な観点がありますが、回帰では決定係数（R二乗値）というものがあります。  
決定係数は0～1の範囲の値で表現され、モデルがそのデータをどれほど説明できているかを表します。  
（例えば決定係数0.6だとそのモデルはデータを60%表現できている）  

決定係数の算出式は以下の通りです。  
$$
R^2 = \frac{\sum_{i=1}^{n}(\hat{y}_i - \bar{y})^2}{\sum_{i=1}^{n}(y_i - \bar{y})^2}
$$

決定係数についての詳細が気になる方向けに、解説と算出方法を以下コードと結果で纏めています。  
下のコードでは y = 20x + 100 という回帰式（モデル）が作成されたと仮定しています。  
xは説明変数、yは目的変数、y_predはxの値と回帰式を使って算出した予測値です。  
  
決定係数は以下の流れで算出されます。  
1. xが変化しようとyが変わらないベースラインモデルを作ります。これはxがどのような値でもyの平均で予測するモデルです。
2. 1.で仮定したベースラインモデルに対して、y（実測値）がどれほど差があるかを計算します。これをST（全変動）といいます。
3. 1.で仮定したベースラインモデルに対して、y_pred（予測値）がどれほど差があるか計算します。これをSR（回帰変動）といいます。
4. SRの和をSTの和で割ったものが決定係数となります。

コードと上記流れを見比べながら、確認してみましょう。  

In [ ]:
# 決定係数を理解する
x = [1,2,3,4,5]
y = [100, 130, 170, 195, 190]

y_pred = [ 20 * i +100 for i in x]

ymean = np.mean(y)

print('y = 20x +100 というモデルが作成されたとする')
print('説明変数:', x)
print('予測値:', y_pred)
print('実測値:', y)
print('実測値平均:', ymean)

# イメージ画像作成用コード
plt.figure(figsize=(8,5))
plt.scatter(x,y_pred,color='orange')
plt.scatter(x,y,color='green')
plt.axhline(ymean, ls=':', lw=1.5, c='red') 
plt.plot([x[3]-0.075,x[3]-0.075],[ymean,y[3]], c='green')
plt.plot([x[3]+0.075,x[3]+0.075],[ymean,y_pred[3]], c='orange')
plt.text(x[3]-0.9, ymean+(y[3]-ymean)/2, '全変動(ST)', fontsize=15, c='green')
plt.text(x[3]+0.1, ymean+(y_pred[3]-ymean)/2, '回帰変動(SR)', fontsize=15, c='orange')
plt.text(x[0], ymean - 5, 'ベースラインモデル（平均）', fontsize=12, c='red')
plt.grid()
plt.title('決定係数の計算について')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

ST = y - ymean
SR = y_pred - ymean
print('全変動(ST)：', ST)
print('回帰変動(SR)：', SR)

R2 = np.sum(SR**2) / np.sum(ST**2)
print('決定係数(R^2)：', R2)

それでは実際に精度を確認してみましょう。  
モデル作成に使った学習データと、モデル作成には使っていない検証データの２つを比較しながら確認します。  

比較するのは「決定係数」の値と「家賃×面積の散布図と回帰直線」のグラフ、「実測値に対する予測誤差」のグラフです。  
※ 本来はもっと確認する項目はありますが、今回は簡単に上記3つのみの比較とします。  

学習データと検証データで決定係数が大幅に異なる場合は、  
学習データに対してフィットしすぎていて他のデータを上手く予測できない過学習が起きている可能性があります。

In [ ]:
# 描画用の関数を作成
def plot_lm(x,y,model):
    print('決定係数 R^2： ', model.score(x, y))
    
    fig, (axL, axR) = plt.subplots(ncols=2, figsize=(15,5))
    axL.scatter(x, y, color = 'orange')
    axL.plot(x, model.predict(x), linestyle="solid", color = 'green')
    axL.set_title('回帰直線')
    axL.set_xlabel('面積')
    axL.set_ylabel('家賃')
    axL.grid()
    
    axR.scatter(y, model.predict(x)-y, color = 'skyblue')
    axR.set_title('予測誤差')
    axR.set_xlabel('実際の家賃')
    axR.set_ylabel('予測値の誤差')
    axR.grid()
    
    plt.show()

In [ ]:
print('モデル関数の回帰係数 : %.3f' %clf.coef_)
print('モデル関数の切片 : %.3f' %clf.intercept_)
print('y= %.3fx + %.3f' % (clf.coef_ , clf.intercept_))
print()

print('==学習データ================================================================')
plot_lm(X_train, Y_train, clf)

print('==検証データ================================================================')
plot_lm(X_test, Y_test, clf)


学習データと検証データで決定係数に大きく差異がでていることはなさそうです。  
また、予測誤差も同じような傾向になっており面積が広い物件ほど誤差が大きくなっていることがわかります。  

まずはこのモデルで任意の値をいれて実際に予測をしてみましょう。  
以下のコードのXの値を変更して、面積によってどのような家賃が予測されているのか確認してください。  

In [ ]:
# 山手線沿線で20平米の物件に住んだら、家賃はいくら？
X = [[20]]
pred = clf.predict(X)
print('面積が{0}平米の時の家賃は{1}円'.format(X[0][0], pred[0]))

---
#### 【Extra3】重回帰分析（たくさんの項目を使って家賃を予測）をやってみる
---

単回帰分析は説明変数が1つだけで、今回は家賃を予測するのに面積を使いました。  
しかし様々な要因で価格が決まる家賃を面積だけで予測するのは現実的ではありません。  
複数の説明変数を使って予測するときは重回帰分析を使用します。  

どの説明変数を使うのかは、業務知識や把握したデータ傾向を考えながら選択することが一般的です。  
もちろんモデルの種類や予測対象によってはすべてのデータを使用することもあります。  
今回は予測に効きそうな変数以外は削除して、一部の説明変数のみを使用していきます。

In [ ]:
# モデルに使わない項目を削除
chintai_tmp = chintai.drop(columns=['建物ID', '物件ID', '管理費', '平米単価', '住所1', '住所2', '住所3', '沿線1',  
                                    '駅名1', '駅徒歩1', '沿線2', '駅名2','駅徒歩2', '沿線3', '駅名3', '駅徒歩3'])

chintai_tmp.head(2)

残した項目のみ使用していきますが、ここで「間取り」と「山手線駅」に注意が必要です。  
この2つの項目はカテゴリ（数値じゃない）となっています。  
カテゴリ値はそのままだと計算することができないので、使用することができません。  
カテゴリ値を使用する方法の1つにone-hot-encoding（ダミー変数化）というものがあります。  
例えば「性別」という項目に「男性」と「女性」という値が入っていた場合、  
one-hot-encodingでこのような値を列の項目として持ち、0/1のフラグとして保持することで、数値計算を可能とします。  

実際に「間取り」と「山手線駅」をone-hot-encodingしてみましょう。  
one-hot-encodingには`pandas.get_dummies`を使用します。

In [ ]:
# カテゴリ値はそのままだと計算できないので0/1のフラグにする（ダミー変数化 or one-hotエンコーディング）
chintai_tmp = pd.get_dummies(chintai_tmp, columns=['間取り', '山手線駅'])

chintai_tmp.head(2)

使用するデータの加工ができたので、ここからは先ほどと同様にデータ分割、モデル作成（学習）、検証データでの予測を行います。  

In [ ]:
# 説明変数をNumpy配列で格納
X2 = chintai_tmp.drop(columns=['家賃']).values
# 目的変数をNumpy配列で格納
Y2 = chintai_tmp['家賃'].values

# 学習用データと検証用データを7:3で分割
X_train2, X_test2, Y_train2, Y_test2 = train_test_split(X2, Y2, train_size = 0.7, test_size = 0.3, random_state = 0)

In [ ]:
# 線形回帰のインスタンスを生成する
clf2 = LinearRegression()

# 学習データから学習してモデルを作る
clf2.fit(X_train2, Y_train2)

In [ ]:
# 検証データをモデルで予測する
Y_pred2 = clf2.predict(X_test2)
Y_pred2[:4]

最後に今回作ったモデルについて、「回帰係数」や「決定係数」と「実測値に対する予測誤差」を確認しましょう。  
また、説明変数を「面積」のみで作成したモデルと結果を比較してみましょう。

In [ ]:
print('モデル関数の回帰係数：')
tmp = [print(f'\t{i}\t\t{j}') for i,j in zip(chintai_tmp.drop(columns=['家賃']).columns, clf2.coef_ )]
print('モデル関数の切片 : %.3f' %clf2.intercept_)


fig, (axL, axR) = plt.subplots(ncols=2, figsize=(15,5))
axL.scatter(Y_train2, clf2.predict(X_train2)-Y_train2, color = 'skyblue')
axL.set_title('学習データ：予測誤差')
axL.set_xlabel('実際の家賃')
axL.set_ylabel('予測値の誤差')
axL.grid()

axR.scatter(Y_test2, clf2.predict(X_test2)-Y_test2, color = 'skyblue')
axR.set_title('検証データ：予測誤差')
axR.set_xlabel('実際の家賃')
axR.set_ylabel('予測値の誤差')
axR.grid()
plt.show()

print('学習データ決定係数 R^2： ', clf2.score(X_train2, Y_train2))
print('検証データ決定係数 R^2： ', clf2.score(X_test2, Y_test2))

■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■END）